## Rastrigin Function

In mathematical optimization, the Rastrigin function is a non-convex function used as a performance test problem for optimization algorithms. It is a typical example of non-linear multimodal function. It was first proposed in 1974 by Rastrigin as a 2-dimensional function and has been generalized by Rudolph. Finding the minimum of this function is a fairly difficult problem due to its large search space and its large number of local minima.[wiki]

In [1]:
import numpy as np
import matplotlib as pltm
import matplotlib.pyplot as plt
from matplotlib import cm 
from mpl_toolkits.mplot3d import Axes3D 

pltm.use('TkAgg')

X = np.linspace(-5.12, 5.12, 100)     
Y = np.linspace(-5.12, 5.12, 100)     
X, Y = np.meshgrid(X, Y) 

Z = (X**2 - 10 * np.cos(2 * np.pi * X)) + (Y**2 - 10 * np.cos(2 * np.pi * Y)) + 20

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.nipy_spectral, linewidth=0.08, antialiased=True)

plt.show()

#plt.savefig('rastrigin_graph.png')

### Homework Assignment

Use the Particle Swarm Optimization Algorithm to optimize the Rastrigin Function

- 2 dimension
- 3 dimension
- 4 dimension

Run the PSO on the Rastrigin Function. Compile the results in table with different iterations, particles, etc. Repeat the results with different implementations of v, c1 (a),c2 (b) as found in the lecture notes in Optimization_PSO slides. Refer to the lecture slides on example of tables required. Exactly what values to choose will be up to you. The objective is to investigate the effect of the variables and attributes of PSO on optimization of the Rastrigin function in multiple dimensions. 

Comment on the results.

Please submit a PDF/Word file presenting your results, comments and discussion.

# Nabil Bachroin (M11107814)

In [2]:
import numpy as np
import pandas as pd

In [3]:
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

In [4]:
class Particle:
    def __init__(self, dim):
        self.position = np.random.uniform(-5.12, 5.12, dim)
        self.velocity = np.random.uniform(-1, 1, dim)
        self.best_position = np.copy(self.position)
        self.best_value = float('inf')

    def update_velocity(self, global_best_position, w, c1, c2):
        r1 = np.random.random(self.position.shape)
        r2 = np.random.random(self.position.shape)
        cognitive_velocity = c1 * r1 * (self.best_position - self.position)
        social_velocity = c2 * r2 * (global_best_position - self.position)
        self.velocity = w * self.velocity + cognitive_velocity + social_velocity

    def update_position(self):
        self.position += self.velocity
        self.position = np.clip(self.position, -5.12, 5.12)  # Keep within bounds

In [5]:
def pso(opt_function, dim, num_particles=30, max_iter=100, w=0.5, c1=2.05, c2=2.05):
    particles = [Particle(dim) for _ in range(num_particles)]
    global_best_value = float('inf')
    global_best_position = None
    best_position_history = []
    
    for it in range(max_iter):
        for particle in particles:
            fitness_value = opt_function(particle.position)
            
            if fitness_value < particle.best_value:
                particle.best_value = fitness_value
                particle.best_position = np.copy(particle.position)
                
            if fitness_value < global_best_value:
                global_best_value = fitness_value
                global_best_position = np.copy(particle.position)
                
        best_position_history.append(np.copy(global_best_position))
        
        for particle in particles:
            particle.update_velocity(global_best_position, w, c1, c2)
            particle.update_position()
            
    return global_best_position, global_best_value, best_position_history

In [6]:
# 2D
best_position_2d, best_value_2d, best_position_history_2d = pso(rastrigin, 2)
print("Best Position 2D:", best_position_2d)
print("Best Value 2D:", best_value_2d)

# 3D
best_position_3d, best_value_3d, best_position_history_3d = pso(rastrigin, 3)
print("Best Position 3D:", best_position_3d)
print("Best Value 3D:", best_value_3d)

# 4D
best_position_4d, best_value_4d, best_position_history_4d = pso(rastrigin, 4)
print("Best Position 4D:", best_position_4d)
print("Best Value 4D:", best_value_4d)

Best Position 2D: [-4.16905316e-08 -1.34823315e-08]
Best Value 2D: 3.801403636316536e-13
Best Position 3D: [ 9.94945823e-01 -4.96059683e-05  5.48142395e-06]
Best Value 3D: 0.9949595838115073
Best Position 4D: [-0.01894698  0.017701    0.99393144 -1.02540053]
Best Value 4D: 2.306933410666119


## Experiments 1
With 100 and 200 iterations, 30 and 50 particles, in each dimension. 

In [7]:
def run_experiment_and_visualize(dim, pso_func, config):
    results = {}
    for exp_config in config:
        particles = exp_config['particles']
        iterations = exp_config['iterations']
        key = f"{particles} particles, {iterations} iterations"
        best_pos, best_val, best_pos_history = pso_func(rastrigin, dim, num_particles=particles, max_iter=iterations)
        results[key] = {
            "best_position": best_pos,
            "best_value": best_val,
            "best_position_history": best_pos_history
        }
    return results

In [8]:
####### Config
experiments_config = [
    {'particles': 30, 'iterations': 100},
    {'particles': 50, 'iterations': 100},
    {'particles': 30, 'iterations': 250},
    {'particles': 50, 'iterations': 250},
]

####### Run
results_2d = run_experiment_and_visualize(2, pso, experiments_config)
results_3d = run_experiment_and_visualize(3, pso, experiments_config)
results_4d = run_experiment_and_visualize(4, pso, experiments_config)

In [9]:
def extract_and_format_results(results):
    data_list = []
    
    for key, value in results.items():
        parts = key.split(', ')
        particles, iterations = parts[0].split(' ')[0], parts[1].split(' ')[0]
        
        if isinstance(value['best_position'], str):
            best_position_str = value['best_position']
        else:
            best_position_str = np.array2string(value['best_position'], precision=5, separator=',', suppress_small=True)
        
        best_value_str = f"{value['best_value']:.2e}"
        
        data_list.append({
            'Particles': particles,
            'Iterations': iterations,
            'Best Position': best_position_str,
            'Best Value': best_value_str
        })
    
    formatted_df = pd.DataFrame(data_list)
    return formatted_df

In [10]:
formatted_results_2d = extract_and_format_results(results_2d)
formatted_results_3d = extract_and_format_results(results_3d)
formatted_results_4d = extract_and_format_results(results_4d)

###### Print results
print("Results for 2D:")
print(formatted_results_2d)
print("\nResults for 3D:")
print(formatted_results_3d)
print("\nResults for 4D:")
print(formatted_results_4d)

Results for 2D:
  Particles Iterations Best Position Best Value
0        30        100     [-0.,-0.]   8.62e-11
1        50        100     [-0., 0.]   0.00e+00
2        30        250     [-0., 0.]   0.00e+00
3        50        250     [-0.,-0.]   0.00e+00

Results for 3D:
  Particles Iterations                 Best Position Best Value
0        30        100  [-0.99496,-0.     ,-0.     ]   9.95e-01
1        50        100                 [ 0.,-0., 0.]   1.67e-10
2        30        250                 [ 0.,-0.,-0.]   0.00e+00
3        50        250                 [ 0.,-0., 0.]   0.00e+00

Results for 4D:
  Particles Iterations                          Best Position Best Value
0        30        100  [-0.99497,-0.00007,-0.     ,-0.00005]   9.95e-01
1        50        100      [1.00194,0.0061 ,0.01862,0.0115 ]   1.11e+00
2        30        250                      [-0., 0., 0., 0.]   0.00e+00
3        50        250                      [ 0.,-0.,-0.,-0.]   2.11e-12


## Experiments 2
With 100 and 200 iterations, 30 and 50 particles, in each dimension. 

The inertia weight (w), cognitive acceleration factor (c1), and social acceleration factor (c2) become dynamic.

In [11]:
def run_experiments(dim, config, pso_func):
    results = {}
    for exp_config in config:
        particles = exp_config['particles']
        iterations = exp_config['iterations']
        key = f"{particles} particles, {iterations} iterations"
        best_pos, best_val, best_pos_history = pso_func(rastrigin, dim, num_particles=particles, max_iter=iterations)
        results[key] = {
            "best_position": best_pos,
            "best_value": best_val,
            "best_position_history": best_pos_history
        }
    return results

In [12]:
def pso_dynamic_params(opt_function, dim, num_particles=30, max_iter=100, w_start=0.9, w_end=0.4, c1_start=1.5, c1_end=2.5, c2_start=1.5, c2_end=2.5):
    particles = [Particle(dim) for _ in range(num_particles)]
    global_best_value = float('inf')
    global_best_position = None
    best_position_history = []
    
    for it in range(max_iter):
        w = w_start - (w_start - w_end) * (it / max_iter)
        c1 = c1_start + (c1_end - c1_start) * (it / max_iter)
        c2 = c2_start + (c2_end - c2_start) * (it / max_iter)
        
        for particle in particles:
            fitness_value = opt_function(particle.position)
            
            if fitness_value < particle.best_value:
                particle.best_value = fitness_value
                particle.best_position = np.copy(particle.position)
                
            if fitness_value < global_best_value:
                global_best_value = fitness_value
                global_best_position = np.copy(particle.position)
                
        best_position_history.append(np.copy(global_best_position))
        
        for particle in particles:
            particle.update_velocity(global_best_position, w, c1, c2)
            particle.update_position()
            
    return global_best_position, global_best_value, best_position_history

In [13]:
####### Config
experiments_config_dynamic = [
    {'particles': 30, 'iterations': 100},
    {'particles': 50, 'iterations': 100},
    {'particles': 30, 'iterations': 250},
    {'particles': 50, 'iterations': 250},
]

In [14]:
def extract_and_format_results(results):
    data_list = []
    
    for key, value in results.items():
        parts = key.split(', ')
        particles, iterations = parts[0].split(' ')[0], parts[1].split(' ')[0]
        
        if isinstance(value['best_position'], str):
            best_position_str = value['best_position']
        else:
            best_position_str = np.array2string(value['best_position'], precision=5, separator=',', suppress_small=True)
        
        best_value_str = f"{value['best_value']:.2e}"
        
        data_list.append({
            'Particles': particles,
            'Iterations': iterations,
            'Best Position': best_position_str,
            'Best Value': best_value_str
        })
    
    return pd.DataFrame(data_list)

In [15]:
###### Run
results_2d_dynamic = run_experiments(2, experiments_config_dynamic, pso_dynamic_params)
results_3d_dynamic = run_experiments(3, experiments_config_dynamic, pso_dynamic_params)
results_4d_dynamic = run_experiments(4, experiments_config_dynamic, pso_dynamic_params)

formatted_results_2d_dynamic = extract_and_format_results(results_2d_dynamic)
formatted_results_3d_dynamic = extract_and_format_results(results_3d_dynamic)
formatted_results_4d_dynamic = extract_and_format_results(results_4d_dynamic)

###### Print results
print("Dynamic Params Results for 2D:")
print(formatted_results_2d_dynamic)
print("\nDynamic Params Results for 3D:")
print(formatted_results_3d_dynamic)
print("\nDynamic Params Results for 4D:")
print(formatted_results_4d_dynamic)

Dynamic Params Results for 2D:
  Particles Iterations      Best Position Best Value
0        30        100            [0.,0.]   2.48e-09
1        50        100  [0.00001,0.     ]   9.61e-09
2        30        250          [ 0.,-0.]   0.00e+00
3        50        250          [-0., 0.]   0.00e+00

Dynamic Params Results for 3D:
  Particles Iterations                 Best Position Best Value
0        30        100     [0.00012,0.00067,0.00151]   5.42e-04
1        50        100  [-0.00002,-0.00021,-0.00022]   1.85e-05
2        30        250     [0.     ,0.     ,0.00002]   1.13e-07
3        50        250                 [-0.,-0.,-0.]   5.89e-09

Dynamic Params Results for 4D:
  Particles Iterations                          Best Position Best Value
0        30        100  [ 0.00671,-0.97156, 1.01699,-0.98817]   3.21e+00
1        50        100  [ 0.99436,-0.00068, 0.0023 , 0.00653]   1.00e+00
2        30        250  [ 0.00037,-0.00008, 0.00006,-0.0001 ]   3.12e-05
3        50        250  [-0.

#### Recap

In [16]:
###### Add Coloumn
formatted_results_2d['Dimensions'] = '2D'
formatted_results_3d['Dimensions'] = '3D'
formatted_results_4d['Dimensions'] = '4D'
formatted_results_2d['Parameter Type'] = 'Static'
formatted_results_3d['Parameter Type'] = 'Static'
formatted_results_4d['Parameter Type'] = 'Static'

formatted_results_2d_dynamic['Dimensions'] = '2D'
formatted_results_3d_dynamic['Dimensions'] = '3D'
formatted_results_4d_dynamic['Dimensions'] = '4D'
formatted_results_2d_dynamic['Parameter Type'] = 'Dynamic'
formatted_results_3d_dynamic['Parameter Type'] = 'Dynamic'
formatted_results_4d_dynamic['Parameter Type'] = 'Dynamic'

###### Concat
all_results = pd.concat([
    formatted_results_2d, formatted_results_3d, formatted_results_4d,
    formatted_results_2d_dynamic, formatted_results_3d_dynamic, formatted_results_4d_dynamic
], ignore_index=True)


all_results_sorted = all_results.sort_values(by=['Dimensions', 'Particles', 'Iterations', 'Parameter Type'])
print(all_results_sorted.to_string(index=False))


Particles Iterations                         Best Position Best Value Dimensions Parameter Type
       30        100                               [0.,0.]   2.48e-09         2D        Dynamic
       30        100                             [-0.,-0.]   8.62e-11         2D         Static
       30        250                             [ 0.,-0.]   0.00e+00         2D        Dynamic
       30        250                             [-0., 0.]   0.00e+00         2D         Static
       50        100                     [0.00001,0.     ]   9.61e-09         2D        Dynamic
       50        100                             [-0., 0.]   0.00e+00         2D         Static
       50        250                             [-0., 0.]   0.00e+00         2D        Dynamic
       50        250                             [-0.,-0.]   0.00e+00         2D         Static
       30        100             [0.00012,0.00067,0.00151]   5.42e-04         3D        Dynamic
       30        100          [-0.99496,

### Visualization

In [17]:
def rastrigin_2D(X, Y):
    A = 10
    return A*2 + (X**2 - A * np.cos(2 * np.pi * X)) + (Y**2 - A * np.cos(2 * np.pi * Y))

In [18]:
def visualize_rastrigin_best_positions(best_positions_history_static, best_positions_history_dynamic, title=""):
    X = np.linspace(-5.12, 5.12, 100)
    Y = np.linspace(-5.12, 5.12, 100)
    X, Y = np.meshgrid(X, Y)
    Z = rastrigin_2D(X, Y)

    fig = plt.figure(figsize=(14, 7))

    ##### Static parameters plot
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.plot_surface(X, Y, Z, cmap=cm.viridis, linewidth=0, antialiased=False, alpha=0.5)
    ax1.scatter(best_positions_history_static[:,0], best_positions_history_static[:,1], rastrigin_2D(best_positions_history_static[:,0], best_positions_history_static[:,1]), color='r', s=50)
    ax1.set_title(f'Static Parameters {title}')
    ax1.set_xlabel('X Axis')
    ax1.set_ylabel('Y Axis')
    ax1.set_zlabel('Z Axis')

    ##### Dynamic parameters plot
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.plot_surface(X, Y, Z, cmap=cm.plasma, linewidth=0, antialiased=False, alpha=0.5)
    ax2.scatter(best_positions_history_dynamic[:,0], best_positions_history_dynamic[:,1], rastrigin_2D(best_positions_history_dynamic[:,0], best_positions_history_dynamic[:,1]), color='b', s=50)
    ax2.set_title(f'Dynamic Parameters {title}')
    ax2.set_xlabel('X Axis')
    ax2.set_ylabel('Y Axis')
    ax2.set_zlabel('Z Axis')

    plt.show()

In [24]:
key_static = '30 particles, 100 iterations'
key_dynamic = '30 particles, 100 iterations'

best_positions_history_static = np.array([history for history in results_2d[key_static]['best_position_history']])
best_positions_history_dynamic = np.array([history for history in results_2d_dynamic[key_dynamic]['best_position_history']])

visualize_rastrigin_best_positions(best_positions_history_static, best_positions_history_dynamic, title="30 Particles, 100 Iterations")

In [25]:
key_static = '50 particles, 100 iterations'
key_dynamic = '50 particles, 100 iterations'

best_positions_history_static = np.array([history for history in results_2d[key_static]['best_position_history']])
best_positions_history_dynamic = np.array([history for history in results_2d_dynamic[key_dynamic]['best_position_history']])

visualize_rastrigin_best_positions(best_positions_history_static, best_positions_history_dynamic, title="50 Particles, 100 Iterations")

In [26]:
key_static = '30 particles, 250 iterations'
key_dynamic = '30 particles, 250 iterations'

best_positions_history_static = np.array([history for history in results_2d[key_static]['best_position_history']])
best_positions_history_dynamic = np.array([history for history in results_2d_dynamic[key_dynamic]['best_position_history']])

visualize_rastrigin_best_positions(best_positions_history_static, best_positions_history_dynamic, title="30 Particles, 250 Iterations")

In [27]:
key_static = '50 particles, 250 iterations'
key_dynamic = '50 particles, 250 iterations'

best_positions_history_static = np.array([history for history in results_2d[key_static]['best_position_history']])
best_positions_history_dynamic = np.array([history for history in results_2d_dynamic[key_dynamic]['best_position_history']])

visualize_rastrigin_best_positions(best_positions_history_static, best_positions_history_dynamic, title="50 Particles, 250 Iterations")

### Done (Bachroin)